In [3]:
import pygui as pgui
pgui.create_interface()

# Example usage:

        # F =>  String: Faction                  'Alloyn'              F=Alloyn
        # D =>  String: Deckname                 'Insane'              D~Forge
        # FB => String: Forgeborn               'Ironbeard'            FB=Ironbeard
        # C  => List:   Cardnames               'Digitize'             C~Digitize
        # A  => List:   Forgeborn - Ability     'Army Commander'       A~'Army Commander'
        # K  => Dict:   Composition             'Robot'                K:Robot > 3

NameError: name 'factionNames' is not defined

In [1]:
from __future__ import print_function
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

from MongoDB.DatabaseManager import DatabaseManager
import GlobalVariables

from pyvis.network import Network
import networkx as nx
import math 
from CardLibrary import Deck
def graphToNet(graph,size=10):
    # Modify the labels of the nodes to include the length of the parents list
    degree_centrality = nx.degree_centrality(graph)
    betweenness_centrality = nx.betweenness_centrality(graph)
    #partition = nx.community.label_propagation_communities(graph)

    metric = betweenness_centrality

    for node, value in metric.items() :
        decimal = value * size * 1000
        graph.nodes[node]['value'] = decimal
        graph.nodes[node]['label'] = node

    for node, data in graph.nodes(data=True):
        num_parents = len(data.get('parents', []))        
        graph.nodes[node]['label'] += f"[{num_parents}]"
        
    net = Network(notebook=True, directed=True, height="1500px", width="2000px", cdn_resources='in_line')    
    net.from_nx(graph)
    net.force_atlas_2based()
    net.show_buttons()
    print("Displaying Graph!")
    #display(net.show('graph.html'))
    return net


In [ ]:
from ast import Global
from CardLibrary import FusionData, Fusion
import GlobalVariables
from MongoDB.DatabaseManager import DatabaseManager
from MyGraph import MyGraph

from soldb import main, parse_arguments
import os

# Define Variables
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

def create_faction_toggle(faction_names, initial_style='info'):
    faction_toggle = widgets.ToggleButtons(
        options=faction_names,
        description='Faction:',
        disabled=False,
        button_style=initial_style,
        tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    )

    def update_button_style(change):
        if change['new'] == 'Alloyin':
            faction_toggle.button_style = 'info'
        elif change['new'] == 'Nekrium':
            faction_toggle.button_style = 'warning'
        elif change['new'] == 'Tempys':
            faction_toggle.button_style = 'danger'
        elif change['new'] == 'Uterra':
            faction_toggle.button_style = 'success'

    faction_toggle.observe(update_button_style, 'value')

    return faction_toggle

def button_load_decks(button):

    arguments = ["--username" , GlobalVariables.username, 
                 "--mode", 'update' ]
                    #"--filter", "C~Biologist" 
                    #"--type", "fuseddeck"  
                    
    args = parse_arguments(arguments)
    main(args)    

def button_on_click(button):
    myDeckA = Deck.lookup(dropdown1.value)
    myDeckB = Deck.lookup(dropdown2.value)

    if myDeckA and myDeckB:
        fusionName = f"{myDeckA.name}_{myDeckB.name}"
        fusionCursor = myDB.find('Fusion', {'name' : fusionName})
        if fusionCursor: 
            for fusion in fusionCursor:
                myFusion = Fusion.from_data(fusion)
                display_graph(myFusion)
        else:
            # Create a new fusion based on the decknames
            newFusionData = FusionData(name=fusionName, myDecks=[myDeckA, myDeckB],tags=['forged'] )
            newFusion = Fusion(newFusionData)
            display_graph(newFusion)
                
    else: 
        for deck in [myDeckA , myDeckB] :
            print(deck)
            if deck:
                myGraph = MyGraph()
                myGraph.create_graph_children(deck)
                net = graphToNet(myGraph.G)
                display(net.show(f"{deck.name}.html"))
def on_username_change(change):
    new_username = change['new']
    if new_username:  # Check that the username is not empty
        #print(f"Changing username -> {new_username}")
        GlobalVariables.username = new_username
        myDB.set_database_name(GlobalVariables.username)
        update_items(factionToggle1, dropdown1)
        update_items(factionToggle2, dropdown2)
    else:
        print("Username cannot be an empty string")

def display_graph(deck):
    myGraph = MyGraph()
    myGraph.create_graph_children(deck)
    net = graphToNet(myGraph.G)
    with out:
        out.clear_output() 
        display(net.show(f"{deck.name}.html"))
def update_items(faction_toggle, dropdown):
    #print(f"Updating items for {faction_toggle.value} , username = {GlobalVariables.username}")
    myDB.set_database_name(GlobalVariables.username)
    deckCursor = myDB.find('Deck', {'faction' : faction_toggle.value})
    deckNames = [deck['name'] for deck in deckCursor]
    dropdown.options = deckNames        

def create_widgets(name) :
    factionToggle = create_faction_toggle(factionNames)
    dropdown = widgets.Dropdown()
    update_items(factionToggle, dropdown)
    factionToggle.observe(lambda change: update_items(factionToggle, dropdown), 'value')
    return factionToggle, dropdown

def create_interface():

    GlobalVariables.username = 'tmind'
    myDB = DatabaseManager(GlobalVariables.username)

factionNames = ['Alloyin', 'Nekrium', 'Tempys', 'Uterra']
deckNames = []

out = widgets.Output()

factionToggle1, dropdown1 = create_widgets('First')       
factionToggle2, dropdown2 = create_widgets('Second')       

button_graph = widgets.Button(description="Show Graph")
button_graph.on_click(button_on_click)

button_load = widgets.Button(description="Load Decks" )
button_load.on_click(button_load_decks)

username = widgets.Text(value='tmind', description='Username:', disabled=False)
username.observe(on_username_change, 'value')

display(username, button_load, factionToggle1, dropdown1, factionToggle2, dropdown2, button_graph)
display(out)    



Text(value='tmind', description='Username:')

Button(description='Load Decks', style=ButtonStyle())

ToggleButtons(button_style='info', description='Faction:', options=('Alloyin', 'Nekrium', 'Tempys', 'Uterra'),…

Dropdown(options=('The Square Proposing Sentry Angels', 'The Raiders of Leading Capitals', 'The Common Wrench …

ToggleButtons(button_style='info', description='Faction:', options=('Alloyin', 'Nekrium', 'Tempys', 'Uterra'),…

...

Fetching Data: 100%|██████████| 163/163 [00:00<?, ?it/s]


Requesting Data from Website: tmind,fuseddeck,


 75%|███████▌  | 19924/26406 [02:54<00:56, 114.00it/s]


Displaying Graph!
The Mutant Constructing Gang.html


None


In [ ]:
from CardLibrary import Fusion, Deck
from MyGraph import MyGraph
from MongoDB.DatabaseManager import DatabaseManager
import GlobalVariables
GlobalVariables.username = 'tmind'
myDB = DatabaseManager(GlobalVariables.username)

fusionCursor = myDB.find('Fusion', {'name' : 'Simian - Sailors'})

deckCursor  = myDB.find('Deck', {'name' : 'The Armoring Studious Power Cult'})

for deck in deckCursor:
    myDeck = Deck.from_data(deck)
    #myDeck.calculate_stats()   
 
    # CardSetName 
# for fusion in fusionCursor:
#     myFusion = Fusion.from_data(fusion)
#     myGraph = MyGraph()
#     myGraph.create_graph_children(myFusion)
#     net = graphToNet(myGraph.G)
#     display(net.show(f"{myFusion.name}.html"))
    
